# Guide for a better C++

## Todo

- [x] Example with string
- [x] Example with matrices
- [x] Eager Evaluation
- [x] Return value optimization
- [x] Cost of virtual functions
- [x] Virtualizing non member functions
- [x] Requiring or prohibiting heap-based objects
- [x] Multiple Dispatching
- [x] Finish More Effective C++
- [ ] Difference between const and constexpr with assmebler explanation
- [ ] Check Combine C++ and C (for personnal purpose not for the guide)
- [ ] Check C++17 specs
- [ ] Check that all snippet of codes compile


**Includes**

In [1]:
#include <iostream>
#include <map>
#include <cstdlib>

## Topics to deal with

### C++ Tips

#### <ins>Lazy evaluation:</ins>

**Principle: Do not evaluate an expression until the results is required.**

This method might save computations whose results are not used.
Here are 4 example of lazy evaluation application areas:

- **Reference counting:** Count references to objects instead of always copying data.
- **Distinguish Reads from Write:** When using reference counting, create new copie of data only when using *operator[]* to write.
- **Lazy Fetching:** When dealing with large objects, read no data from object until a specific attribute is asked for Read/Write.
- **Lazy Evaluation:** When making operations on structure, store the operation in a structure and defer computation until result is required.

##### <ins>Example 1:</ins>

In [2]:
std::string s1 = "Hello";
std::string s2 = s1; //Calls string copy ctor which performs an allocation 
                //then a strcopy to copy data
                //(s1 and s2 have their own copy of the value)

printf("Address of s1 data: %p\n", s1.data());
printf("Address of s2 data: %p\n", s2.data());

Address of s1 data: 0x7f4bbce47040
Address of s2 data: 0x7f4bbce47060


This is how data is stored, however it is not needed to have data duplicated until s2 is modified.

<div style="text-align:center"><img src="https://i.imgur.com/zxaUBIm.png" /></div>

One method is to implement a struct nested in String that holds a *referenceCounter* and avoid data duplication until it is really needed.
Here is a graphical representation of this model:


<div style="text-align:center"><img src="https://i.imgur.com/Ov7zKM4.png" /></div>

Snippet of code representing this model.

In [3]:
class String
{
    public:
        struct StringValue {
            //Init data and strcopy initValue inside
            //Set refCount to 1
            StringValue(const char* initValue)
            : m_pData(new char[strlen(initValue)]),
              m_sRefCount(1)
            {
            }

            //Delete data
            ~StringValue() { delete[] m_pData; }

            size_t m_sRefCount;
            char* m_pData;        
        };


        String(const char* data="")
        : m_pValue(new StringValue(data))
        {    
        }

        //Destroy StringValue
        ~String()
        {
            delete m_pValue;
        }

        //Copy constructor => Do the same for operator=
        String(const String& rhs)
        {
            std::cout << "COPY CTOR" << std::endl;
            if (m_pValue == rhs.m_pValue)    return;

            //By doing this, we postpone data duplication
            // data evaluation consisting on: creating a new char[] and calling strcopy
            if (m_pValue && --m_pValue->m_sRefCount == 0)    delete m_pValue;

            m_pValue = rhs.m_pValue;
            m_pValue->m_sRefCount++;
        }

        //Evaluation of duplication operation
        void EvaluateDuplication()
        {
            //Decrease reference counter
            m_pValue->m_sRefCount--;    

            //Creates its own StringValue
            m_pValue = new StringValue(m_pValue->m_pData);
        }

        //Writing operation: data duplication must be evaluated
        void ToLowerCase()
        {
            if (m_pValue->m_sRefCount > 1)
                EvaluateDuplication();

            //... LowerCase operation
        }

        StringValue* m_pValue;
};

In [4]:
String S1("Hello");
String S2 = S1;

//show data addr
printf("Reference count of S1 Value: %zu\n", S1.m_pValue->m_sRefCount);
printf("Address of S1 data: %p\n", S1.m_pValue->m_pData);
printf("Address of S2 data: %p\n", S2.m_pValue->m_pData);

//Here let's make a modification on S2
S2.ToLowerCase();
std::cout << "\nPerformed modification on S2 !\n\n";

//We can see its address is now different also ref count changed
printf("Reference count of S1 Value: %zu\n", S1.m_pValue->m_sRefCount);
printf("Address of S2 data: %p\n", S2.m_pValue->m_pData);

COPY CTOR
Reference count of S1 Value: 2
Address of S1 data: 0x555fac8af980
Address of S2 data: 0x555fac8af980

Performed modification on S2 !

Reference count of S1 Value: 1
Address of S2 data: 0x555faca47500


##### <ins>Example 2:</ins>

Here is a class representing a 2D vector:

In [5]:
struct Vec2
{
    float m_x;
    float m_y;
    
    Vec2(float x, float y)
    : m_x(x),
      m_y(y)
    {
          std::cout << "CTOR" << std::endl;
    }
    
    Vec2 operator+(const Vec2& rhs) const
    {
        std::cout << "SUM" << std::endl;
        return Vec2(m_x + rhs.m_x, m_y + rhs.m_y);
    }
    
    float dot(const Vec2& rhs)
    {
        std::cout << "DOT" << std::endl;
        return m_x * rhs.m_x + m_y * rhs.m_y;
    }
};

Now, let's considerate a function foo that sums two Vec2 and compute the dot product with a third Vec2.

In [6]:
Vec2 A(1,2);
Vec2 B(3, 4);
Vec2 C(2.0f, 3.0f);
(A + B).dot(C);

CTOR
CTOR
CTOR
SUM
CTOR
DOT


Using lazy evaluation, it is possible to save a call to Vec2 ctor&dtor because there is no need for Vec2::operator+ to be called.

In [7]:
//Create a structure that will hold sum operand and
//compute result only if needed
struct addVec2
{
    struct LazyVec2
    {

        float m_x;
        float m_y;

        LazyVec2(float x, float y)
        : m_x(x),
          m_y(y)
        {
              std::cout << "CTOR" << std::endl;
        }

        //Let's change Vec2::operator+
        //Do not forget to change declaration
        addVec2  operator+(const LazyVec2& rhs)
        {
            return addVec2(*this, rhs);
        }

        float dot(const LazyVec2& rhs)
        {
            std::cout << "DOT" << std::endl;
            return m_x * rhs.m_x + m_y * rhs.m_y;
        }
    };


    addVec2(const LazyVec2& lhs, const LazyVec2& rhs)
    : a(lhs),
      b(rhs)
    {
        std::cout << "CTOR of holding struct" << std::endl;
    }

    //Here we save Vec2 ctor/dtor call
    float dot(const LazyVec2& v)
    {
        return a.dot(v) + b.dot(v);
    }

    //Evaluates sum 
    LazyVec2 Eval()
    {
        std::cout << "SUM" << std::endl;
        return LazyVec2(a.m_x + b.m_x, a.m_y + b.m_y);
    }

    LazyVec2 a;
    LazyVec2 b;
};

Notice that sum is not evaluated (since we do not need result to be computed) and no intermediate Vec2 is created to hold the result of the struct.

In [8]:
addVec2::LazyVec2 A(1,2);
addVec2::LazyVec2 B(3, 4);
addVec2::LazyVec2 C(2.0f, 3.0f);
(A + B).dot(C);

CTOR
CTOR
CTOR
CTOR of holding struct
DOT
DOT


#### <ins>Eager evaluation:</ins> 

**Principle: Doing things before they are needed to minimize further computations.**

When you expect computation to be resquested frequently, it is possible to reduce the cost requests by precomputing the results.

##### <ins>Example 1</ins>

Let's consider a database containing information about clients and you expect many queries on clients age. It is possible to use local caching to store information about age and avoid querying the database all the time.

In [9]:
int getClientAge(const std::string& clientName)
{
    //Executed once
    static std::map<std::string, int> mapAges;
    
    auto it = mapAges.find(clientName);
    
    //Data not contained in map
    if (it == mapAges.end())
    {
        int age = std::rand() % 40 + 20;//Query database
        
        //Add queried data to local cache
        mapAges[clientName] = age;
        
        std::cout << "Caching " + clientName + " data !" << std::endl;

        return age;
    }
    //Queried data is already in local cache
    //save a call to database
    else
        return it->second;
    
}

In [10]:
std::cout << getClientAge("Fedina") << std::endl;
std::cout << getClientAge("Seb") << std::endl;
std::cout << getClientAge("Max") << std::endl;
std::cout << getClientAge("Théo") << std::endl;
std::cout << getClientAge("Victor") << std::endl;
std::cout << getClientAge("Xavier") << std::endl;

std::cout << "\nRetrieving cached data.\n\n";

std::cout << getClientAge("Fedina") << std::endl;
std::cout << getClientAge("Seb") << std::endl;
std::cout << getClientAge("Max") << std::endl;
std::cout << getClientAge("Théo") << std::endl;
std::cout << getClientAge("Victor") << std::endl;
std::cout << getClientAge("Xavier") << std::endl;

Caching Fedina data !
47
Caching Seb data !
20
Caching Max data !
25
Caching Théo data !
41
Caching Victor data !
46
Caching Xavier data !
34
Retrieving cached data.

47
20
25
41
46
34


##### <ins>Example 2</ins>

Suppose we have a *Vector* containing numeric data and we expect min, max to be asked frequently. It is better to precompute them when data is modified than having to compute them each time they are required.